### 1b. Determining when the house is cheating. (20pts)
Now that you know the cheating dealer's strategy, you decide that you want to catch the dealer in the act.  Play 1000 games of dice with the dealer, using the forward algorithm to determine when the dealer is cheating.  Recall that you now don't have access to the dealer's state, only the rolls!  Assume that the dealer starts using the fair die.  Report the probability that the dealer is using the loaded die at each of the 1000 rolls.

HINT: In the notation of Murphy, you are looking for $P(z_t=j|\mathbf{x}_{1:t})=\alpha_t(j)$, where $z_t=j\in\{0,1\}$ are the possible states and $\mathbf{x}_{1:t}$ are all the die rolls up to the current time.  The two step formula for computing this quantity can be found in Murphy Eqs. 17.44-17.47.


# Homework 3 (Due Oct. 11)

## 1. The occasionally dishonest casino

Imagine that you are employed by the Montana State Gaming Board, and you have received an anonymous tip suggesting that the dealer in the dice games that frequently occur at the Oxford is cheating.  In particular, the dealer is surreptitiously switching from a fair die to a loaded one (and back) with a certain probability.  In this game, the ante is a doller.  you choose a number, the dealer rolls the die, and if it matches you get back 3 times your ante.  Otherwise, you lose your ante.  

You examine security camera footage from which you extracted the following dataset, with *states* describing whether the die was loaded, and rolls being the outcome of each die roll.

In [1]:
import pickle
import matplotlib
import matplotlib.pyplot as plt

states,rolls = pickle.load(open('oxford.p','rb'))
plt.plot(states)
plt.show()
print (len(states))


<Figure size 640x480 with 1 Axes>

1000


### 1a. Training a HMM (20pts)
Assume that the dealer follows a Markov model
$$
P(S_{t+1}=s_k|S_t=s_j) = A_{jk}, s_j\in\{0,1\},s_k\in\{0,1\},
$$
where $s_j=0$ corresponds to use of the fair die, while $s_j=1$ means the die is loaded.  The outcome of the die roll follows a categorical distribution conditioned on which die was being used
$$
P(R_t=r|S_t=s_j) = E_{jr}, r_t\in\{0,1,2,3,4,5\}.
$$
Note that for convenience, the dealer is using a six-sided, zero-indexed die.

Train your model of the dealer's ruse by using your observations to find the maximum likelihood estimators for the transition matrix $A$ and the emission probabilities $E$.  

HINT: because the states are observed, training $A$ is equivalent to training a normal Markov model.  Similarly, training $E$ is equivalant to training a Naive Bayes model, where the state is the class and the die roll is the feature.

In [2]:
import numpy as np
m = len(states)

A = np.zeros((2,2))
E = np.zeros((2,6))
#! Compute the parameters of the transition and emission matrices.
lock = False 
counter = 0
num_roll_0=0
num_roll_1=0
previous_state=states[0]
for state , roll in zip(states,rolls):
    E[state][roll] +=1
    if counter > 0:
        A[state] [previous_state] +=1
    previous_state = state
    counter +=1
for row in E:
    row /=row.sum()
A /= A.sum(axis=1)[:,np.newaxis]
print (E)


[[0.16623377 0.17792208 0.14155844 0.16883117 0.18701299 0.15844156]
 [0.0826087  0.06521739 0.12608696 0.09130435 0.11304348 0.52173913]]


In [3]:
print (A)

[[0.98959688 0.01040312]
 [0.03913043 0.96086957]]


In [4]:
def likelihood(roll,state):
    return E[state,roll]

def prediction(state):
    return np.dot(state,A)

possible_states = [0,1]
possible_rolls = [0,1,2,3,4,5]

alpha = [np.array([1.0,0.0])]
current_state = 0
current_roll = np.random.choice(possible_rolls,p=E[0])

states = [current_state]
rolls = [current_roll]

for g in range(1000):
        
    ### This is the dealer's internal state, you can't use this information!
    current_state = np.random.choice(possible_states,p=A[current_state])
    states.append(current_state)
    ###
    
    ### This is the dealer's emission, you *can* use this information
    current_roll = np.random.choice(possible_rolls,p=E[current_state])
    rolls.append(current_roll)
    ###
    
    #! Implement the recursive forward algorithm to determine the probability 
    #! of being in the fair or loaded state for all rolls
    last_S_Probability = alpha[g]
    Predictions = prediction(last_S_Probability)  
    like_hood = np.array(likelihood(current_roll,possible_states))
    Possible_alpha = like_hood * Predictions
    aloha_row =Possible_alpha / Possible_alpha.sum()
    #print (aloha_row)
    alpha.append(aloha_row)
   

In [5]:
def MyPrediction(alpha_):
    Prediction_list = []
    for i in alpha_ :
        if i[0]> i[1]:
            Prediction_list.append(0)
        else:
            Prediction_list.append(1)
    return Prediction_list

def ToString(s):
    Mystring=''
    for state in s:
        if state == 0:
            Mystring +='F'# when the casino play with fair die add 'F' to the string
        else:
            Mystring +='-'# when the casino play with loaded die add '-' to the string
    return Mystring
Prediction_list = MyPrediction(alpha)
correct_prediction=0
for i,m in enumerate(states):
    if Prediction_list[i]== states[i]:
        correct_prediction +=1
percentage = (correct_prediction/ float (len (states))) * 100  
print ("this model is correct in percentage = " , "%.2f" % percentage, "%")


this model is correct in percentage =  90.31 %


In [6]:
# printing 2 strings for the actual states and predicted list where 'F' is playing fair and '-' otherwise
s1= ToString(states)
s2= ToString(Prediction_list)
n=0
for i in range (10):
    print ('actual states:',s1[n:(n+100)])
    print ('Prediction   :',s2[n:(n+100)])
    print ("___________________________________________________________________________________________________________________")
    n=n+100

actual states: FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF------FFFFF
Prediction   : FFFFFFFF-FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF--FFFF
___________________________________________________________________________________________________________________
actual states: FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF---------------FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF----FFFFFFFFF
Prediction   : FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF-FF-FF-F-F-----FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF-FFFFFFFFF
___________________________________________________________________________________________________________________
actual states: FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF
Prediction   : FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF--FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF-F
________________________________________________________________________

### 1C (\*). Beating the house (10pts)
Describe a strategy for beating the house given all the information determined from the above two problems.  Show that your strategy will, in general, make money after 1000 rolls, either by mathematical proof, or by direct simulation. 

In [7]:
# functions
# this function return true win I when an flase if I lose
def Game(Roll, bet):
    if Roll == bet:
        return True
    else:
        return False
# this function return the Last state string 
def Laststate(F,L):
    if F > L:
        return 'Fair'
    else:
        return 'Loaded'

In [18]:
#This function to call it for 100 times
def loopfor100():
    
    # One of the simplest approaches is to put more money when dealer use loaded dice 
    #  and choose the number that has the high percentage.
    # let assume I will paly for 1000 roll

    #And I have 1000$ for it as minimum bet for each roll is 1$
    # If I win, I take 3 times what I bet on. Or lose my bet
    Mymoney = 1000
    #but I can put 10$ more when I feel dialer is using loaded dice
    mybet = 1   # some times I will but it 10
    alpha = [np.array([1.0,0.0])]
    current_state = 0


    # before the first roll let's play fair and guess equally
    Myguesses= [np.random.choice(possible_rolls,p=E[0])]
    current_roll = np.random.choice(possible_rolls,p=E[0])

    # game start for the first roll
    Mymoney -=mybet
    if Game(current_roll,Myguesses[0]):
        Mymoney += (mybet * 3)

    states = [current_state]
    rolls = [current_roll]
    last_state = 'Fair'
    for g in range(1000):
        # here I assume that if it fair or loaded dice, he is more likely will use it again this time
        last_state = Laststate(alpha[g][0],alpha[g][1])
        # so I will build my guess on the last state he was in and raise my bet
        if last_state == 'Fair':
            Myguess = np.random.choice(possible_rolls,p=E[0])
            mybet = 1
        else:
            Myguess = 5 #np.random.choice(possible_rolls,p=E[1])
            mybet = 10
        Myguesses.append(Myguess)
        Mymoney -= mybet

        ### This is the dealer's internal state, you can't use this information!
        current_state = np.random.choice(possible_states,p=A[current_state])
        states.append(current_state)
        ###

        ### This is the dealer's emission, you *can* use this information
        current_roll = np.random.choice(possible_rolls,p=E[current_state])
        rolls.append(current_roll)
        ###

        if Game(current_roll,Myguess):
            Mymoney += (mybet * 3)

        #! Implement the recursive forward algorithm to determine the probability 
        #! of being in the fair or loaded state for all rolls
        last_S_Probability = alpha[g]
        Predictions = prediction(last_S_Probability)  
        like_hood = np.array(likelihood(current_roll,possible_states))
        Possible_alpha = like_hood * Predictions
        aloha_row =Possible_alpha / Possible_alpha.sum()
        #print (aloha_row)
        alpha.append(aloha_row)
    myconter = 0
    for i,j in zip(rolls,Myguesses):
        if i == j:
            myconter +=1
    #print (myconter )
    #print (Mymoney)
    if Mymoney > 1000:
        return True
    else:
        return False
winning = 0 # holed number of winning 

for i in range(100):
    if loopfor100():
       winning +=1
print ("After applying my strategic for 100 times which is raise the bet 10$ on number 5 ")
print ("if I feel that last role was by the loaded die the winning time was : ", winning)


After applying my strategic for 100 times which is raise the bet 10$ on number 5 
if I feel that last role was by the loaded die the winning time was :  58
